In [1]:
const plantuml = require('plantuml');
const tslab = require("tslab");

In [5]:
import jwt_decode from "jwt-decode";

## Howto offer VC to Holder

<div class="container16x9"><iframe src="https://www.youtube.com/embed/6PsQOLE6D8k" class="responsive-iframe" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></div>

We already know about classical triangle of roles.
It is **user centric** 
- holder - user that own a VCs
- issuer - party that issue a VC
- verifier - could aks holder for a VCs to Verify

We need a **protocol** that could proof the identity and **autorship** for every role.
As we know our identities is our keypairs so we should sign some entities to proof our identity 
Affinidi SDK offer transport agnostic protocols for excanges based on JWT tokens


In [3]:
const plantumlEncoder = require('plantuml-encoder')
const tslab = require("tslab");
const diagram = plantumlEncoder.encode(`
@startuml
digraph F {
  fontname="Helvetica,Arial,sans-serif"

	node [fontname="Helvetica,Arial,sans-serif"]
	edge [fontname="Helvetica,Arial,sans-serif"]

	node [shape = circle];
	holder [lablel="holder" shape = doublecircle]
		
	issuer [lablel="issuer"]
	verifier [lablel="verifier"]
	
	verifier -> holder[label = "request for sharing"]
	holder -> verifier[label = "request for sharing"]
	issuer -> holder [label = "request for offer"];
	holder -> issuer [label = "response for offer"];
	issuer -> holder [label = "VC"];
}
@enduml
`);
const urldiagram = 'http://www.plantuml.com/plantuml/svg/' + diagram
tslab.display.html(`<img src="${urldiagram}"/>`)


With request/responce for offer holder and issuer could identify each other

In [5]:
const ShareOfferDiagram = plantumlEncoder.encode(`
@startuml
title Affinidi: Affinidi: Offer VC Flow
actor "SDK (Holder)" as holder
actor  "SDK (Issuer)" as client
participant  "Affinidi Issuer" as issuer
participant "Affinidi Registry" as registry

client -> issuer: BuildCredentialOffer(offeredCredentials)
issuer -> client : Issuer Service: 200 [credentialOfferRequest]
note over client, issuer: credentialOfferRequest signing and credentialOfferRequestToken generated
issuer -> holder: 200[credentialOfferRequestToken]

note over holder, client
NOTE:
Verification of the Issuer signature on the credentialOfferRequestToken
1. Resolve Issuer DID (get publicKey)
2. Verify Issuer signature (using publicKey)
end note 
holder -> registry: resolveDid(IssuerDid)
registry -> holder: 200 [DidDocument]
holder -> client: send back(CredentialOfferResponseToken)

note over client, holder
Verification of the Holder signature on the CredentialOfferResponseToken
1. Resolve Holder DID (get publicKey)
2. Verify Holder signature (using publicKey)
end note
client -> registry: resolveDid(HolderDid)
registry -> client: 200[DidDocument]
client -> issuer: CreateSignedCredential(data, metadata, **requesterDid**)
issuer -> client: 200[signedCredentials]
client -> holder: 200 [Token wirth credentials inside]

@enduml
`);
const ShareOfferDiagramUrl = 'http://www.plantuml.com/plantuml/svg/' + ShareOfferDiagram
tslab.display.html(`<img src="${ShareOfferDiagramUrl}"/>`)


<img src="http://www.plantuml.com/plantuml/svg/VLJ1Rjim3BthAuYUx20hkXvz6Bh56rOKM86cw2N8GPLeXwWYPPAS8dy_MR8x9Kvp2HYHZuzy9CtkdEVMTrl5F7c5yBrfI9EaAhCUcmOjl2pWbpBlZ0jlB5mjwmSeVXibqPPNm1rieZsu1_-zSzte5ufGUxOBT2HenxKFKIC9fCWOID4y2lmVzuGjEMyFCT8EVnXBoU7Bjm5TmO-Eb5nObE6Tk8ez5AR_nUpPbIm1UkYGf1gAWINQFGciuFRc1bRYEDSJ_ktG-JNJnYEOVOXF-Fb72USHuAZLf5lWMduIycpUK4EB6YttABCIayYnejK5x9fbPINCV1p0dyVddnLxGKiD2Ux9Q30D-0sERVVrST_P0DVn_G8H-teTHkACsd_0w_iQYXOzxBfNHU813oMxlOP8U9Xo59thnSY2CUWIgsUfyhxlST9Li29RJNBOhc2LRFIVQ0IhuAsDwBQX_dMMBsbHWUl9NhbuAnQdJRgTqGvZboKxC-LXtyy9cSvYAkGbZbp90NzPoGd9vqfcvt5Mo9Hg8kGeKhziHpfEhorq5ZPr6Mh9hwkGtFCvRD7pPCrcDYrFP9lDfjUNsDn98fTp7iqtdSexMR_9zZHyHBGZYU4Exe88_UVj7m00"/>

In [1]:
const { createWallet } = require('@affinidi/wallet-node-sdk')
const walletFactory = createWallet('AffinityCore')


Lets setup wallets

In [ ]:
const accessApiKey = '<your key>'

In [2]:
const options = {
    env: 'prod',
    accessApiKey,
}
  const issuerWallet = await walletFactory.createWallet(options, 'P@55word!!!')
  const holderWallet = await walletFactory.createWallet(options, 'P@55word!!!')

## Offer Request
- It is holder agnostic
- it help toidentify issuer
- it is share information about credentials that we want to issue
What we want to offer

In [3]:
    const offeredCredentials = [
      {
        type: 'ContentLike',
        renderInfo:{},
      }]
    
     const callbackUrl = 'https://vp-service.com/callback'
     
     const credentialOfferRequestToken = await issuerWallet.generateCredentialOfferRequestToken(
      offeredCredentials,
      {
        callbackUrl,
      },
    )
     credentialOfferRequestToken

eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NksifQ.eyJpbnRlcmFjdGlvblRva2VuIjp7ImNhbGxiYWNrVVJMIjoiaHR0cHM6Ly92cC1zZXJ2aWNlLmNvbS9jYWxsYmFjayIsIm9mZmVyZWRDcmVkZW50aWFscyI6W3sidHlwZSI6IkNvbnRlbnRMaWtlIiwicmVuZGVySW5mbyI6e319XX0sImV4cCI6MTY2MjgzNzg5OTQ0NSwidHlwIjoiY3JlZGVudGlhbE9mZmVyUmVxdWVzdCIsImp0aSI6IjNmYTdhN2FkMTFiNWQ0YWMiLCJpc3MiOiJkaWQ6ZWxlbTpFaURVYldBUXNYOVk1S21aOVloYlJ1N0NyNGp5Y2k0WnVmdmVCM0FDUVlONzdBO2VsZW06aW5pdGlhbC1zdGF0ZT1leUp3Y205MFpXTjBaV1FpT2lKbGVVcDJZMGRXZVZsWVVuQmlNalJwVDJsS2FtTnRWbWhrUjFWcFRFTktjbUZYVVdsUGFVbHFZMGhLY0dKWFJubGxVMGx6U1cxR2MxcDVTVFpKYTFaVVRXcFZNbE41U2praUxDSndZWGxzYjJGa0lqb2laWGxLUVZreU9YVmtSMVkwWkVOSk5rbHRhREJrU0VKNlQyazRkbVI2VG5CYVF6VjJZMjFqZG1NeVZtcGtXRXB3WkVocmRtUnFTV2xNUTBwM1pGZEtjMkZYVGt4YVdHdHBUMngwTjBsdGJHdEphbTlwU1ROQ2VXRlhNV2hqYm10cFRFTktNV015Um01YVUwazJTVzVPY0ZveU5YQmliV05wVEVOS01HVllRbXhKYW05cFZUSldhbU5FU1RGT2JYTjRWbTFXZVdGWFduQlpNa1l3WVZjNWRWTXlWalZOYWtGNFQwTkpjMGx1UWpGWmJYaHdXVEIwYkdWVmFHeGxRMGsyU1dwQmVVMUVXWHBPVkdjeFRrZFpNbGxxWkd4UFZGRXlUakpGTlUxWFJUR

In [6]:
jwt_decode(credentialOfferRequestToken)

{
  interactionToken: {
    callbackURL: 'https://vp-service.com/callback',
    offeredCredentials: [ [Object] ]
  },
  exp: 1662837899445,
  typ: 'credentialOfferRequest',
  jti: '3fa7a7ad11b5d4ac',
  iss: 'did:elem:EiDUbWAQsX9Y5KmZ9YhbRu7Cr4jyci4ZufveB3ACQYN77A;elem:initial-state=eyJwcm90ZWN0ZWQiOiJleUp2Y0dWeVlYUnBiMjRpT2lKamNtVmhkR1VpTENKcmFXUWlPaUlqY0hKcGJXRnllU0lzSW1Gc1p5STZJa1ZUTWpVMlN5SjkiLCJwYXlsb2FkIjoiZXlKQVkyOXVkR1Y0ZENJNkltaDBkSEJ6T2k4dmR6TnBaQzV2Y21jdmMyVmpkWEpwZEhrdmRqSWlMQ0p3ZFdKc2FXTkxaWGtpT2x0N0ltbGtJam9pSTNCeWFXMWhjbmtpTENKMWMyRm5aU0k2SW5OcFoyNXBibWNpTENKMGVYQmxJam9pVTJWamNESTFObXN4Vm1WeWFXWnBZMkYwYVc5dVMyVjVNakF4T0NJc0luQjFZbXhwWTB0bGVVaGxlQ0k2SWpBeU1EWXpOVGcxTkdZMllqZGxPVFEyTjJFNU1XRTFZelEzT0dVM05XRmtabUV3WlRnd1ptWmhNVFl6TkRnd1lXWmlPRFU1TXpjNE5qbG1OVFV6TVRSaE15SjlMSHNpYVdRaU9pSWpjbVZqYjNabGNua2lMQ0oxYzJGblpTSTZJbkpsWTI5MlpYSjVJaXdpZEhsd1pTSTZJbE5sWTNBeU5UWnJNVlpsY21sbWFXTmhkR2x2Ymt0bGVUSXdNVGdpTENKd2RXSnNhV05MWlhsSVpYZ2lPaUl3TW1FNU9HWXdOamd5WXpVd1pXWmhZems1TnpreE5ES

## Offer Response
Lets holder to send response 

In [20]:
    const credentialOfferResponseToken = await holderWallet.createCredentialOfferResponseToken(
      credentialOfferRequestToken,
    )
    credentialOfferResponseToken

eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NksifQ.eyJpbnRlcmFjdGlvblRva2VuIjp7ImNhbGxiYWNrVVJMIjoiaHR0cHM6Ly92cC1zZXJ2aWNlLmNvbS9jYWxsYmFjayIsInNlbGVjdGVkQ3JlZGVudGlhbHMiOlt7InR5cGUiOiJDb250ZW50TGlrZSIsInJlbmRlckluZm8iOnt9fV19LCJleHAiOjE2NjI4MzkyMjk2NjcsInR5cCI6ImNyZWRlbnRpYWxPZmZlclJlc3BvbnNlIiwianRpIjoiM2ZhN2E3YWQxMWI1ZDRhYyIsImF1ZCI6ImRpZDplbGVtOkVpRFViV0FRc1g5WTVLbVo5WWhiUnU3Q3I0anljaTRadWZ2ZUIzQUNRWU43N0E7ZWxlbTppbml0aWFsLXN0YXRlPWV5SndjbTkwWldOMFpXUWlPaUpsZVVwMlkwZFdlVmxZVW5CaU1qUnBUMmxLYW1OdFZtaGtSMVZwVEVOS2NtRlhVV2xQYVVscVkwaEtjR0pYUm5sbFUwbHpTVzFHYzFwNVNUWkphMVpVVFdwVk1sTjVTamtpTENKd1lYbHNiMkZrSWpvaVpYbEtRVmt5T1hWa1IxWTBaRU5KTmtsdGFEQmtTRUo2VDJrNGRtUjZUbkJhUXpWMlkyMWpkbU15Vm1wa1dFcHdaRWhyZG1ScVNXbE1RMHAzWkZkS2MyRlhUa3hhV0d0cFQyeDBOMGx0Ykd0SmFtOXBTVE5DZVdGWE1XaGpibXRwVEVOS01XTXlSbTVhVTBrMlNXNU9jRm95TlhCaWJXTnBURU5LTUdWWVFteEphbTlwVlRKV2FtTkVTVEZPYlhONFZtMVdlV0ZYV25CWk1rWXdZVmM1ZFZNeVZqVk5ha0Y0VDBOSmMwbHVRakZaYlhod1dUQjBiR1ZWYUd4bFEwazJTV3BCZVUxRVdYcE9WR2N4VGtkWk1sbHFaR3hQVkZFeVRqSkZOVTFYU

In [7]:
jwt_decode(credentialOfferResponseToken)

{
  interactionToken: {
    callbackURL: 'https://vp-service.com/callback',
    selectedCredentials: [ [Object] ]
  },
  exp: 1662837898338,
  typ: 'credentialOfferResponse',
  jti: '3fa7a7ad11b5d4ac',
  aud: 'did:elem:EiDUbWAQsX9Y5KmZ9YhbRu7Cr4jyci4ZufveB3ACQYN77A;elem:initial-state=eyJwcm90ZWN0ZWQiOiJleUp2Y0dWeVlYUnBiMjRpT2lKamNtVmhkR1VpTENKcmFXUWlPaUlqY0hKcGJXRnllU0lzSW1Gc1p5STZJa1ZUTWpVMlN5SjkiLCJwYXlsb2FkIjoiZXlKQVkyOXVkR1Y0ZENJNkltaDBkSEJ6T2k4dmR6TnBaQzV2Y21jdmMyVmpkWEpwZEhrdmRqSWlMQ0p3ZFdKc2FXTkxaWGtpT2x0N0ltbGtJam9pSTNCeWFXMWhjbmtpTENKMWMyRm5aU0k2SW5OcFoyNXBibWNpTENKMGVYQmxJam9pVTJWamNESTFObXN4Vm1WeWFXWnBZMkYwYVc5dVMyVjVNakF4T0NJc0luQjFZbXhwWTB0bGVVaGxlQ0k2SWpBeU1EWXpOVGcxTkdZMllqZGxPVFEyTjJFNU1XRTFZelEzT0dVM05XRmtabUV3WlRnd1ptWmhNVFl6TkRnd1lXWmlPRFU1TXpjNE5qbG1OVFV6TVRSaE15SjlMSHNpYVdRaU9pSWpjbVZqYjNabGNua2lMQ0oxYzJGblpTSTZJbkpsWTI5MlpYSjVJaXdpZEhsd1pTSTZJbE5sWTNBeU5UWnJNVlpsY21sbWFXTmhkR2x2Ymt0bGVUSXdNVGdpTENKd2RXSnNhV05MWlhsSVpYZ2lPaUl3TW1FNU9HWXdOamd5WXpVd1pXWmhZems1TnpreE5

# Verification of Response
Now issuer could validate a holder

In [21]:
const result = await issuerWallet.verifyCredentialOfferResponseToken(
      credentialOfferResponseToken,
      credentialOfferRequestToken,
    )

result

{
  isValid: true,
  did: 'did:elem:EiAtmNkxOhWOhTCIxORy62LpFrNZQtVM7g6oDaxmDDlqmA;elem:initial-state=eyJwcm90ZWN0ZWQiOiJleUp2Y0dWeVlYUnBiMjRpT2lKamNtVmhkR1VpTENKcmFXUWlPaUlqY0hKcGJXRnllU0lzSW1Gc1p5STZJa1ZUTWpVMlN5SjkiLCJwYXlsb2FkIjoiZXlKQVkyOXVkR1Y0ZENJNkltaDBkSEJ6T2k4dmR6TnBaQzV2Y21jdmMyVmpkWEpwZEhrdmRqSWlMQ0p3ZFdKc2FXTkxaWGtpT2x0N0ltbGtJam9pSTNCeWFXMWhjbmtpTENKMWMyRm5aU0k2SW5OcFoyNXBibWNpTENKMGVYQmxJam9pVTJWamNESTFObXN4Vm1WeWFXWnBZMkYwYVc5dVMyVjVNakF4T0NJc0luQjFZbXhwWTB0bGVVaGxlQ0k2SWpBeU5XVTFNREkzTW1FM09UQTVPV1ZtTVRNNFpXRXpPVEF4WWpJeE5USTBOakEwWldNMVpETXhZakZqWVdJNFlqbGlaamt5WmpWak1EY3hNVEV5WlRRNE1DSjlMSHNpYVdRaU9pSWpjbVZqYjNabGNua2lMQ0oxYzJGblpTSTZJbkpsWTI5MlpYSjVJaXdpZEhsd1pTSTZJbE5sWTNBeU5UWnJNVlpsY21sbWFXTmhkR2x2Ymt0bGVUSXdNVGdpTENKd2RXSnNhV05MWlhsSVpYZ2lPaUl3TXpJeE5XVXlZV1ZtWTJKbE1qQmpZV1ExWWpSbVl6Vm1PR1JsT0RRd01USTJObUl3T0dRellXWTBOekpsTWpFNU5ERXhZekptT1RGaFlqVTFZamMyTVdVaWZWMHNJbUYxZEdobGJuUnBZMkYwYVc5dUlqcGJJaU53Y21sdFlYSjVJbDBzSW1GemMyVnlkR2x2YmsxbGRHaHZaQ0k2V3lJamNISnBiV0

## Issue VC for Holder
Now we could issue a credential for holder

In [10]:
const jsonSchema = 'https://schema.affinidi.com/ContentLikeV1-0.json'
const jsonContext = 'https://schema.affinidi.com/ContentLikeV1-0.jsonld'
const id = `claimId:${(Math.random() + 1).toString(36).substring(2)}`
const unsignedVC = {
    '@context': ['https://www.w3.org/2018/credentials/v1', jsonContext],
    id,
    type: ['VerifiableCredential', 'ContentLike'],
    holder: {
      id: 'placeholder'
    },
    credentialSubject: {
      data: {
        '@type': ['VerifiableCredential', 'ContentLike'],
        url: 'https://www.youtube.com/watch?v=owbkzvLhblk',
        date: new Date().toISOString(),
        like: true,
        score: 10
      },
    },
    credentialSchema: {
      id:  jsonSchema,
      type: 'JsonSchemaValidator2018',
    },
    issuanceDate: new Date().toISOString(),
    expirationDate: '2065-09-10T00:00:00.000Z',
  }   
const signedCredentials = await issuerWallet.signCredentials(
      credentialOfferResponseToken,
      [unsignedVC],
    )
signedCredentials

[
  {
    '@context': [
      'https://www.w3.org/2018/credentials/v1',
      'https://schema.affinidi.com/ContentLikeV1-0.jsonld'
    ],
    id: 'claimId:3b1426be69d03bc1',
    type: [ 'VerifiableCredential', 'ContentLike' ],
    holder: { id: 'did:elem:EiAtmNkxOhWOhTCIxORy62LpFrNZQtVM7g6oDaxmDDlqmA' },
    credentialSubject: { data: [Object] },
    issuanceDate: '2022-09-10T19:21:57.743Z',
    expirationDate: '2065-09-10T00:00:00.000Z',
    issuer: 'did:elem:EiDUbWAQsX9Y5KmZ9YhbRu7Cr4jyci4ZufveB3ACQYN77A;elem:initial-state=eyJwcm90ZWN0ZWQiOiJleUp2Y0dWeVlYUnBiMjRpT2lKamNtVmhkR1VpTENKcmFXUWlPaUlqY0hKcGJXRnllU0lzSW1Gc1p5STZJa1ZUTWpVMlN5SjkiLCJwYXlsb2FkIjoiZXlKQVkyOXVkR1Y0ZENJNkltaDBkSEJ6T2k4dmR6TnBaQzV2Y21jdmMyVmpkWEpwZEhrdmRqSWlMQ0p3ZFdKc2FXTkxaWGtpT2x0N0ltbGtJam9pSTNCeWFXMWhjbmtpTENKMWMyRm5aU0k2SW5OcFoyNXBibWNpTENKMGVYQmxJam9pVTJWamNESTFObXN4Vm1WeWFXWnBZMkYwYVc5dVMyVjVNakF4T0NJc0luQjFZbXhwWTB0bGVVaGxlQ0k2SWpBeU1EWXpOVGcxTkdZMllqZGxPVFEyTjJFNU1XRTFZelEzT0dVM05XRmtabUV3WlRnd1ptWmhNVFl6T